# Word2Vec (Negative Sampling)

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [33]:
import nltk
from nltk.corpus import semcor, stopwords

nltk.download('semcor')
nltk.download('stopwords')

[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'NVIDIA GeForce RTX 2080 with Max-Q Design')

In [35]:
semcor.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

## 1. Load data

In [36]:
#1. tokenization
corpus = semcor.sents()

In [37]:
stop_words = set(stopwords.words('english'))
corpus = [[word for word in sent if word.lower() not in stop_words] for sent in corpus]

# Remove punctuation from corpus
from string import punctuation
corpus = [[word for word in sent if word not in punctuation] for sent in corpus]

# Remove empty sentences
corpus = [sent for sent in corpus if len(sent) > 0]

# Remove sentences with less than 5 words
corpus = [sent for sent in corpus if len(sent) >= 5]

# Remove sentences with more than 20 words
corpus = [sent for sent in corpus if len(sent) <= 20]


In [38]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [39]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}

In [40]:
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1  

In [41]:
index2word = {v:k for k, v in word2index.items()}

## 2. Prepare train data

In [42]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            
            #outside words (window size on both sides of the center word)
            outside_start =  i - window_size
            outside_end =  i + window_size + 1 # +1 because the end index is exclusive

            # outside = []
            # loop through the outside words, append to the list 'outside'
            for j in range(outside_start, outside_end):
                if i != j:  # Skip the center word
                    outside= word2index[doc[j]]
                    skipgrams.append([center, outside])

                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [43]:
x.shape  #batch_size, 1

(2, 1)

In [44]:
x

array([[  173],
       [21241]])

In [45]:
y.shape  #batch_size 1

(2, 1)

## 3. Negative Sampling

### Unigram distribution

$$P(w)=U(w)^{3/4}/Z$$

In [46]:
z = 0.001

In [47]:
#count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

292890

In [48]:
vocabs

['bateau',
 'Span',
 'pennies',
 'farming',
 'capital',
 'McAuliffe',
 'gyrations',
 'nods',
 'quickening',
 'buses',
 'sweltering',
 'counteract',
 'labeling',
 'ministered',
 'collar',
 'tree',
 'vitriolic',
 'Action',
 'arm-levitation',
 'hints',
 'myriad',
 'surrounded',
 'Iijima',
 'objectors',
 'malformations',
 'mitigates',
 'Carsten',
 'affirm',
 'Loop',
 'moody',
 'unachievable',
 'Capable',
 'une',
 'Ackerly',
 'video',
 'Higher',
 'death-like',
 'du',
 'Milstein',
 'sinusoids',
 'meaningless',
 'occupation',
 'Tom',
 'slacks',
 'Consonantal',
 'Pezza',
 'Indochina',
 'Stuart',
 'Bucks',
 'excised',
 'Vacancy',
 'evensong',
 'motive',
 'Du',
 'distastefully',
 'sound',
 'silver-painted',
 'mercilessly',
 'article',
 'squadrons',
 'meant',
 'permissibility',
 'Dexter',
 'hydrogen',
 'App',
 'smug',
 'nettled',
 'Bloch',
 'rode',
 'circulatory',
 'goodies',
 'straightway',
 'pay',
 'engineer',
 'mandrel',
 'girl-friend',
 'pirouette',
 'pectorals',
 'immediate',
 'ashore',
 'nu

$$P(w)=U(w)^{3/4}/Z$$

In [49]:
unigram_table = []

for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)
    
Counter(unigram_table)

Counter({'``': 40,
         "''": 40,
         "'s": 31,
         'would': 19,
         'one': 19,
         'said': 15,
         "n't": 14,
         'could': 13,
         'time': 12,
         'two': 11,
         'man': 10,
         'like': 10,
         'first': 10,
         'may': 10,
         'must': 9,
         'new': 9,
         'made': 9,
         'also': 9,
         'way': 8,
         'Mr.': 8,
         'many': 8,
         'years': 8,
         'back': 8,
         'even': 8,
         'much': 8,
         'people': 7,
         'long': 7,
         'good': 7,
         'year': 7,
         'work': 7,
         'get': 7,
         'make': 7,
         'f': 7,
         'little': 7,
         'world': 6,
         'well': 6,
         'go': 6,
         'us': 6,
         'still': 6,
         'men': 6,
         'know': 6,
         'day': 6,
         'see': 6,
         'never': 6,
         'last': 6,
         'came': 6,
         'take': 6,
         'life': 6,
         'might': 6,
         'left': 5,

## 4. Model

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [50]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

In [51]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [52]:
batch_size = 2
x, y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [53]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [54]:
y_tensor[1]

tensor([28264])

In [55]:
neg_samples[1]

tensor([ 8790, 28807,  6070, 19074, 35513])

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [56]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)
        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)
        

In [57]:
#test your model
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size)

In [58]:
loss = model(x_tensor, y_tensor, neg_samples)

In [59]:
loss

tensor(5.3982, grad_fn=<NegBackward0>)

## 5. Training

In [60]:
import time

def log_epoch(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [63]:
start = time.time()
num_epochs = 1000
window_size = 5

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus, window_size)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    # Log epoch time
    epoch_mins, epoch_secs = log_epoch(start, time.time())
    #print the loss
    if (epoch + 1) % 100 == 0:
        print(f"Epoch: {epoch + 1} | Loss: {loss:.6f} | Time: {epoch_mins}m {epoch_secs}s")

Epoch: 100 | Loss: 0.866487 | Time: 1m 17s
Epoch: 200 | Loss: 1.709664 | Time: 2m 54s
Epoch: 300 | Loss: 1.586200 | Time: 4m 33s
Epoch: 400 | Loss: 1.197748 | Time: 6m 10s
Epoch: 500 | Loss: 2.297212 | Time: 7m 46s
Epoch: 600 | Loss: 4.942214 | Time: 9m 23s
Epoch: 700 | Loss: 2.161587 | Time: 11m 0s
Epoch: 800 | Loss: 1.236760 | Time: 12m 37s
Epoch: 900 | Loss: 1.544715 | Time: 14m 15s
Epoch: 1000 | Loss: 2.506096 | Time: 15m 56s


## 6. Plot the embeddings

In [64]:
vocabs

['bateau',
 'Span',
 'pennies',
 'farming',
 'capital',
 'McAuliffe',
 'gyrations',
 'nods',
 'quickening',
 'buses',
 'sweltering',
 'counteract',
 'labeling',
 'ministered',
 'collar',
 'tree',
 'vitriolic',
 'Action',
 'arm-levitation',
 'hints',
 'myriad',
 'surrounded',
 'Iijima',
 'objectors',
 'malformations',
 'mitigates',
 'Carsten',
 'affirm',
 'Loop',
 'moody',
 'unachievable',
 'Capable',
 'une',
 'Ackerly',
 'video',
 'Higher',
 'death-like',
 'du',
 'Milstein',
 'sinusoids',
 'meaningless',
 'occupation',
 'Tom',
 'slacks',
 'Consonantal',
 'Pezza',
 'Indochina',
 'Stuart',
 'Bucks',
 'excised',
 'Vacancy',
 'evensong',
 'motive',
 'Du',
 'distastefully',
 'sound',
 'silver-painted',
 'mercilessly',
 'article',
 'squadrons',
 'meant',
 'permissibility',
 'Dexter',
 'hydrogen',
 'App',
 'smug',
 'nettled',
 'Bloch',
 'rode',
 'circulatory',
 'goodies',
 'straightway',
 'pay',
 'engineer',
 'mandrel',
 'girl-friend',
 'pirouette',
 'pectorals',
 'immediate',
 'ashore',
 'nu

In [65]:
banana = torch.LongTensor([word2index['banana']])
banana

tensor([35507])

In [66]:
banana_embed_c = model.embedding_center(banana)
banana_embed_o = model.embedding_outside(banana)
banana_embed   = (banana_embed_c + banana_embed_o) / 2
banana_embed

tensor([[ 0.2904, -0.3527]], grad_fn=<DivBackward0>)

In [67]:
banana_embed_o

tensor([[ 0.4904, -0.3780]], grad_fn=<EmbeddingBackward0>)

In [68]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    
    return embed[0][0].item(), embed[0][1].item()

In [69]:
get_embed('fruit')

(0.810792088508606, -1.6347533464431763)

In [70]:
get_embed('cat')

(1.095782995223999, 0.536375880241394)

In [71]:
get_embed('dog')

(0.9461292624473572, -0.9850912094116211)

In [72]:
get_embed('banana')

(0.29036882519721985, -0.3527320623397827)

## 7. Cosine similarity

In [73]:
banana = get_embed('banana')
banana

(0.29036882519721985, -0.3527320623397827)

In [74]:
fruit = get_embed('fruit')
fruit

(0.810792088508606, -1.6347533464431763)

In [75]:
cat = get_embed('cat')
cat

(1.095782995223999, 0.536375880241394)

In [76]:
np.array(banana) @ np.array(cat)

0.12898425056742013

In [77]:
#more formally is to divide by its norm
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

print(cosine_similarity(np.array(banana), np.array(cat)))
print(cosine_similarity(np.array(banana), np.array(fruit)))

0.23140595916762477
0.9740508303792865
